# Recomendaciones de Películas con Word2Vec
En este notebook, vamos a utilizar un dataset de recomendaciones de películas para entrenar un modelo **Word2Vec**. Este modelo nos permitirá encontrar películas similares a partir de las listas de películas que los usuarios han visto.

In [3]:
import pandas as pd
from urllib.request import urlopen
from transformers import AutoTokenizer
from gensim.models import Word2Vec

## Carga del Dataset
Vamos a utilizar el dataset de **MovieLens**, que contiene información sobre las películas vistas por los usuarios. Descargaremos los datos y los prepararemos para el entrenamiento del modelo.

In [5]:
# Descargar el dataset de MovieLens
movies_data = pd.read_csv('archivos/CursoGenAI_UAL_02_embeddingsWord2Vec_movies.csv')
ratings_data = pd.read_csv('archivos/CursoGenAI_UAL_02_embeddingsWord2Vec_ratings.csv')

# Mostrar las primeras filas del dataset de películas y ratings para asegurarnos de que la carga fue exitosa
print("Primeras filas de las películas:")
print(movies_data.head())

print("Primeras filas de las calificaciones:")
print(ratings_data.head())


Primeras filas de las películas:
   movieId                               title   
0        1                    Toy Story (1995)  \
1        2                      Jumanji (1995)   
2        3             Grumpier Old Men (1995)   
3        4            Waiting to Exhale (1995)   
4        5  Father of the Bride Part II (1995)   

                                        genres  
0  Adventure|Animation|Children|Comedy|Fantasy  
1                   Adventure|Children|Fantasy  
2                               Comedy|Romance  
3                         Comedy|Drama|Romance  
4                                       Comedy  
Primeras filas de las calificaciones:
   userId  movieId  rating  timestamp
0       1        1     4.0  964982703
1       1        3     4.0  964981247
2       1        6     4.0  964982224
3       1       47     5.0  964983815
4       1       50     5.0  964982931


## Preprocesamiento de Datos
Ahora, vamos a preparar el dataset para el entrenamiento del modelo Word2Vec. Vamos a agrupar las películas vistas por cada usuario para formar secuencias que representen las listas de reproducción de cada usuario.

In [6]:
# Agrupar las películas vistas por cada usuario
user_movie_sequences = ratings_data.groupby('userId')['movieId'].apply(list).tolist()

# Filtrar listas que contengan más de una película
filtered_sequences = [sequence for sequence in user_movie_sequences if len(sequence) > 1]

# Mostrar algunas secuencias de películas
print("Primeras secuencias de películas vistas por usuarios:")
print(filtered_sequences[:5])

Primeras secuencias de películas vistas por usuarios:
[[1, 3, 6, 47, 50, 70, 101, 110, 151, 157, 163, 216, 223, 231, 235, 260, 296, 316, 333, 349, 356, 362, 367, 423, 441, 457, 480, 500, 527, 543, 552, 553, 590, 592, 593, 596, 608, 648, 661, 673, 733, 736, 780, 804, 919, 923, 940, 943, 954, 1009, 1023, 1024, 1025, 1029, 1030, 1031, 1032, 1042, 1049, 1060, 1073, 1080, 1089, 1090, 1092, 1097, 1127, 1136, 1196, 1197, 1198, 1206, 1208, 1210, 1213, 1214, 1219, 1220, 1222, 1224, 1226, 1240, 1256, 1258, 1265, 1270, 1275, 1278, 1282, 1291, 1298, 1348, 1377, 1396, 1408, 1445, 1473, 1500, 1517, 1552, 1573, 1580, 1587, 1617, 1620, 1625, 1644, 1676, 1732, 1777, 1793, 1804, 1805, 1920, 1927, 1954, 1967, 2000, 2005, 2012, 2018, 2028, 2033, 2046, 2048, 2054, 2058, 2078, 2090, 2093, 2094, 2096, 2099, 2105, 2115, 2116, 2137, 2139, 2141, 2143, 2161, 2174, 2193, 2253, 2268, 2273, 2291, 2329, 2338, 2353, 2366, 2387, 2389, 2395, 2406, 2414, 2427, 2450, 2459, 2470, 2478, 2492, 2502, 2528, 2529, 2542, 2571, 

## Entrenamiento del Modelo Word2Vec
Usaremos las secuencias de películas vistas por los usuarios para entrenar el modelo Word2Vec, lo que nos permitirá identificar relaciones entre películas según cómo se ven juntas.

In [7]:
# Entrenar el modelo Word2Vec con las secuencias de películas
embedding_model = Word2Vec(
    filtered_sequences, vector_size=32, window=5, negative=15, min_count=1, workers=4
)

# Guardar el modelo entrenado para su uso posterior
embedding_model.save("movie_recommendation_word2vec.model")

## Evaluación del Modelo
Ahora que el modelo está entrenado, vamos a probarlo encontrando las películas más similares a una película dada.

In [8]:
# Definir una película específica por su ID
target_movie_id = 1  # Por ejemplo, ID de la película "Toy Story (1995)"

# Obtener las películas más similares a la película especificada
similar_movies = embedding_model.wv.most_similar(positive=[target_movie_id])
print("Películas similares a la película con ID 1 (Toy Story):")
for movie_id, similarity in similar_movies:
    movie_title = movies_data.loc[movies_data['movieId'] == movie_id, 'title'].values[0]
    print(f"Título: {movie_title} - Similaridad: {similarity}")


Películas similares a la película con ID 1 (Toy Story):
Título: Twelve Monkeys (a.k.a. 12 Monkeys) (1995) - Similaridad: 0.9950618743896484
Título: Ace Ventura: When Nature Calls (1995) - Similaridad: 0.9945407509803772
Título: Babe (1995) - Similaridad: 0.9940406680107117
Título: GoldenEye (1995) - Similaridad: 0.9934611916542053
Título: Sense and Sensibility (1995) - Similaridad: 0.9923250675201416
Título: Leaving Las Vegas (1995) - Similaridad: 0.9913697838783264
Título: Heat (1995) - Similaridad: 0.9897852540016174
Título: Jumanji (1995) - Similaridad: 0.9893587827682495
Título: Casino (1995) - Similaridad: 0.9862034916877747
Título: Sabrina (1995) - Similaridad: 0.9851760268211365


# Ejercicios

Esta es la documentación de word2vec: https://radimrehurek.com/gensim/models/word2vec.html

### Ejercicio 1

¿Qué vector de embedding tienen las siguientes películas?
- Paper, The (1994)
- Ruby in Paradise (1993)

In [12]:
# En el archivo de películas, buscamos el ID de las películas:
target_movie_id = 371
similar_movies = embedding_model.wv.most_similar(positive=[target_movie_id])
print(f"Películas similares a la película con ID {target_movie_id}:")
for movie_id, similarity in similar_movies:
    movie_title = movies_data.loc[movies_data['movieId'] == movie_id, 'title'].values[0]
    print(f"Título: {movie_title} - Similaridad: {similarity}")
print("Embedding: ")
print(embedding_model.wv[target_movie_id].shape)


target_movie_id = 523
similar_movies = embedding_model.wv.most_similar(positive=[target_movie_id])
print(f"Películas similares a la película con ID {target_movie_id}:")
for movie_id, similarity in similar_movies:
    movie_title = movies_data.loc[movies_data['movieId'] == movie_id, 'title'].values[0]
    print(f"Título: {movie_title} - Similaridad: {similarity}")

Películas similares a la película con ID 371:
Título: Naked Gun 33 1/3: The Final Insult (1994) - Similaridad: 0.996755063533783
Título: Timecop (1994) - Similaridad: 0.9963018298149109
Título: Richie Rich (1994) - Similaridad: 0.996067225933075
Título: It Could Happen to You (1994) - Similaridad: 0.9958513379096985
Título: Red Rock West (1992) - Similaridad: 0.995254635810852
Título: Flintstones, The (1994) - Similaridad: 0.9937567114830017
Título: Wes Craven's New Nightmare (Nightmare on Elm Street Part 7: Freddy's Finale, A) (1994) - Similaridad: 0.9929433465003967
Título: Carlito's Way (1993) - Similaridad: 0.9922701716423035
Título: Beverly Hills Cop III (1994) - Similaridad: 0.9921116232872009
Título: Speed (1994) - Similaridad: 0.9918348789215088
Embedding: 
(32,)
Películas similares a la película con ID 523:
Título: Shadow, The (1994) - Similaridad: 0.9944896697998047
Título: Sirens (1994) - Similaridad: 0.9939926266670227
Título: Secret Garden, The (1993) - Similaridad: 0.9934

### Ejercicio 2
¿Cómo podría entrenar Word2Vec con el archivo del quijote?
Impleméntalo y obtén las palabras más similares a:
- Quijote
- Sancho
- Panza
- Molino

In [13]:
# Paso 1: Leer el archivo .txt
with open('archivos/quijote.txt', 'r', encoding='utf-8') as file:
    text_content = file.read()

# Inicializar el tokenizer
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

# Paso 2: Dividir el texto en bloques más pequeños de 1000 caracteres
max_length = 1000  # Ajustar según tus necesidades
text_chunks = [text_content[i:i + max_length] for i in range(0, len(text_content), max_length)]

# Paso 3: Tokenizar cada fragmento de texto
token_chunks = [tokenizer.tokenize(chunk) for chunk in text_chunks]

# Verificar la cantidad de fragmentos y algunos ejemplos para asegurar que la tokenización es correcta
print(f"El texto se ha dividido en {len(token_chunks)} fragmentos.")
print("Ejemplo de fragmento tokenizado:", token_chunks[0][:10])  # Mostrar los primeros 10 tokens del primer fragmento

# Paso 4: Entrenar el modelo Word2Vec utilizando las listas de tokens
embedding_model = Word2Vec(
    token_chunks, vector_size=32, window=3, negative=15, min_count=1, workers=4
)

c:\Users\Marcos\Programas\Python3.8\lib\site-packages\transformers\tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


El texto se ha dividido en 2098 fragmentos.
Ejemplo de fragmento tokenizado: ['el', 'ing', '##eni', '##oso', 'hidalgo', 'don', 'qui', '##jo', '##te', 'de']


In [14]:
array_palabras = ["Quijote","Sancho","Panza","Molino"]
for palabra in array_palabras:
    palabra_codificada = tokenizer.tokenize(palabra)
    print(f'{palabra}\t{embedding_model.wv.most_similar(positive=palabra_codificada)}')

Quijote	[('fernando', 0.7667080760002136), ('antonio', 0.7060130834579468), ('luis', 0.6953632235527039), ('pedro', 0.6646602749824524), ('diego', 0.6328753232955933), ('-', 0.6298264265060425), ('lorenzo', 0.6247361302375793), ('juan', 0.6104427576065063), ('##de', 0.5775159597396851), ('alvaro', 0.5738563537597656)]
Sancho	[('teresa', 0.7062196731567383), ('-', 0.6502576470375061), ('di', 0.6499964594841003), (':', 0.6021126508712769), ('pan', 0.5912923812866211), ('rep', 0.568871259689331), ('##lic', 0.5557653903961182), ('##jo', 0.5527087450027466), ('barber', 0.5388609170913696), ('ya', 0.5344620943069458)]
Panza	[('teresa', 0.7032684087753296), ('lan', 0.619231641292572), ('belle', 0.5934838652610779), ('grande', 0.5648461580276489), ('##ste', 0.5349306464195251), ('noble', 0.5310447812080383), ('gibraltar', 0.5260404348373413), ('##dan', 0.5075939297676086), ('rus', 0.4996979832649231), ('##gr', 0.49007904529571533)]
Molino	[('lan', 0.7117639780044556), ('fine', 0.70562273263931

## Conclusiones
En este notebook, utilizamos un modelo Word2Vec para analizar las relaciones entre películas basadas en las preferencias de los usuarios. Este enfoque puede ayudar a mejorar los sistemas de recomendación de películas, sugiriendo títulos que son más relevantes para los intereses del usuario.